In [27]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

RANDOM_SEED = 42

### Specify Paths

In [3]:
data_path = './data/data.csv'
model_path = './model/model.keras'

### Load training Data

In [44]:
df = pd.read_csv(data_path)
df

,label,T-31_P0x,T-31_P0y,T-31_P0z,T-31_P1x,T-31_P1y,T-31_P1z,T-31_P2x,T-31_P2y,T-31_P2z,...,T-0_P17z,T-0_P18x,T-0_P18y,T-0_P18z,T-0_P19x,T-0_P19y,T-0_P19z,T-0_P20x,T-0_P20y,T-0_P20z
0,1.0,0.019263,0.048485,1.879826e-10,0.021847,0.045686,-0.000466,0.023458,0.040864,-0.000754,...,-0.002054,0.018095,0.031726,-0.002793,0.018241,0.029022,-0.003340,0.018489,0.026397,-0.003745
1,1.0,0.019556,0.050036,8.835470e-11,0.022346,0.047041,-0.000667,0.024055,0.042453,-0.001244,...,-0.002287,0.014547,0.033535,-0.002999,0.013894,0.031098,-0.003538,0.013433,0.028690,-0.003940
2,1.0,0.018768,0.048917,1.314620e-10,0.021628,0.046195,-0.000730,0.023458,0.041890,-0.001376,...,-0.002788,0.014871,0.034806,-0.003463,0.014359,0.032344,-0.003894,0.014039,0.029862,-0.004246
3,1.0,0.018849,0.047390,1.854362e-10,0.020983,0.043579,-0.000378,0.022037,0.038595,-0.000825,...,-0.001994,0.017994,0.031706,-0.002658,0.018098,0.029269,-0.003122,0.018314,0.027038,-0.003465
4,1.0,0.019334,0.051645,4.650522e-10,0.021398,0.046072,-0.000526,0.021826,0.040004,-0.000989,...,-0.001346,0.019643,0.028969,-0.001886,0.019848,0.026893,-0.002317,0.020085,0.024965,-0.002617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2.0,0.117613,0.207473,-1.483001e-09,0.128249,0.199260,0.000232,0.134922,0.190433,-0.003371,...,-0.022481,0.086482,0.202958,-0.029665,0.091590,0.214580,-0.030832,0.096191,0.221307,-0.030682
249,2.0,0.123608,0.227080,1.106370e-09,0.129836,0.212964,-0.006354,0.133496,0.196217,-0.012512,...,-0.016783,0.089593,0.206081,-0.023232,0.094405,0.216915,-0.025102,0.098668,0.221289,-0.025625
250,2.0,0.121935,0.213034,-1.277578e-10,0.130696,0.203816,-0.003471,0.137413,0.192418,-0.008966,...,-0.015905,0.085989,0.199631,-0.018179,0.089220,0.207823,-0.018065,0.091371,0.209584,-0.018382
251,2.0,0.114530,0.236877,3.225093e-10,0.123477,0.225779,-0.011929,0.127019,0.209489,-0.020565,...,-0.021921,0.090403,0.201474,-0.026315,0.095781,0.207042,-0.026407,0.099563,0.210762,-0.026558


In [46]:
tst_df = df[-100:]
tst_df

,label,T-31_P0x,T-31_P0y,T-31_P0z,T-31_P1x,T-31_P1y,T-31_P1z,T-31_P2x,T-31_P2y,T-31_P2z,...,T-0_P17z,T-0_P18x,T-0_P18y,T-0_P18z,T-0_P19x,T-0_P19y,T-0_P19z,T-0_P20x,T-0_P20y,T-0_P20z
153,1.0,0.020185,0.040631,2.370710e-11,0.021690,0.039581,-0.000256,0.022794,0.037837,-0.000504,...,-0.001091,0.016835,0.030546,-0.001399,0.016712,0.029207,-0.001608,0.016682,0.027989,-0.001763
154,1.0,0.023472,0.039262,3.777600e-11,0.024815,0.038758,0.000126,0.025877,0.037207,0.000165,...,-0.000609,0.014048,0.030535,-0.000763,0.013787,0.029222,-0.000801,0.013622,0.028100,-0.000832
155,1.0,0.022358,0.039499,-2.078866e-11,0.023751,0.039044,-0.000112,0.024809,0.037635,-0.000262,...,-0.000923,0.017269,0.029382,-0.001135,0.017279,0.028202,-0.001255,0.017364,0.027142,-0.001340
156,1.0,0.021666,0.043901,5.887427e-11,0.023140,0.043265,-0.000168,0.024295,0.041744,-0.000362,...,-0.000531,0.014447,0.031011,-0.000750,0.014344,0.029750,-0.000879,0.014240,0.028718,-0.000973
157,1.0,0.022316,0.040744,9.591507e-11,0.023689,0.040640,-0.000267,0.025018,0.039710,-0.000475,...,-0.000743,0.009971,0.033640,-0.000999,0.009566,0.032557,-0.001176,0.009237,0.031584,-0.001302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,2.0,0.117613,0.207473,-1.483001e-09,0.128249,0.199260,0.000232,0.134922,0.190433,-0.003371,...,-0.022481,0.086482,0.202958,-0.029665,0.091590,0.214580,-0.030832,0.096191,0.221307,-0.030682
249,2.0,0.123608,0.227080,1.106370e-09,0.129836,0.212964,-0.006354,0.133496,0.196217,-0.012512,...,-0.016783,0.089593,0.206081,-0.023232,0.094405,0.216915,-0.025102,0.098668,0.221289,-0.025625
250,2.0,0.121935,0.213034,-1.277578e-10,0.130696,0.203816,-0.003471,0.137413,0.192418,-0.008966,...,-0.015905,0.085989,0.199631,-0.018179,0.089220,0.207823,-0.018065,0.091371,0.209584,-0.018382
251,2.0,0.114530,0.236877,3.225093e-10,0.123477,0.225779,-0.011929,0.127019,0.209489,-0.020565,...,-0.021921,0.090403,0.201474,-0.026315,0.095781,0.207042,-0.026407,0.099563,0.210762,-0.026558


In [49]:
X_train, y_train = tst_df.iloc[:,1:], tst_df['label']
X_test, y_test = tst_df.iloc[:,1:], tst_df['label']

In [50]:
import seaborn as sns
from sklearn import metrics
import matplotlib.pyplot as plt

sns.set(rc={"figure.figsize":(12, 10)})

def print_stats(y_pred):
    print(f'accuracy:\t {metrics.accuracy_score(y_test, y_pred)}')
    print(f'precision:\t {metrics.precision_score(y_test, y_pred, average="macro")}')
    print(f'recall:\t\t {metrics.recall_score(y_test, y_pred, average="macro")}')
    print(f'f-score:\t {metrics.f1_score(y_test, y_pred, average="macro")}')
    cf_matrix = metrics.confusion_matrix(y_test, y_pred)
    cf_matrix_n = cf_matrix.astype('float') / cf_matrix.sum(axis=1)[:, np.newaxis]
    sns.heatmap(cf_matrix, cmap='Greens', annot=True)
    plt.ylabel('Actual', size=14)
    plt.xlabel('Predicted', size=14)
    plt.title('confusion matrix', size=18)
    plt.show(block=False)
    sns.heatmap(cf_matrix_n, cmap='Blues', annot=True)
    plt.ylabel('Actual', size=14)
    plt.xlabel('Predicted', size=14)
    plt.title('confusion matrix percentages', size=18)
    plt.show(block=False)

In [ ]:
from sklearn.linear_model import LogisticRegression

log_base = LogisticRegression()
log_base.fit(X_train, y_train)